In [5]:
import numpy as np
import pandas as pd
data = pd.read_csv('./train.csv')
# print('查看数据信息：列名、非空个数、类型等')
# print(data.info())
# print('-'*30)
# print('查看数据摘要')
# print(data.describe())
# print('-'*30)
# print('查看离散数据分布')
# print(data.describe())
train_data=data.sample(frac=1)
# print("Sample info:")
# print(train_data.info())

In [17]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomTreesEmbedding, RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline

X=train_data.drop(['id','target'],axis=1)
y=train_data['target']

n_estimator = 10
# 生成样本集
#print(X)
# 将样本集分成测试集和训练集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train, y_train, test_size=0.5)

# 基于GBDT监督变换
grd = GradientBoostingClassifier(n_estimators=n_estimator)
grd.fit(X_train, y_train)
# 得到OneHot编码
grd_enc = OneHotEncoder(categories='auto')
temp = grd.apply(X_train)
np.set_printoptions(threshold=np.inf)  
grd_enc.fit(grd.apply(X_train)[:, :, 0])
#print(grd_enc.get_feature_names()) # 查看每一列对应的特征
# 使用OneHot编码作为特征，训练LR
grd_lm = LogisticRegression(solver='lbfgs', max_iter=1000)
grd_lm.fit(grd_enc.transform(grd.apply(X_train_lr)[:, :, 0]), y_train_lr)
# # 使用LR进行预测
test_data = pd.read_csv('./test.csv')
test_entry = test_data.drop(['id'],axis=1)
y_pred_grd_lm = grd_lm.predict_proba(grd_enc.transform(grd.apply(test_entry)[:, :, 0]))[:, 1]
# y_pred_grd_lm[y_pred_grd_lm<0.5]=0
# y_pred_grd_lm[y_pred_grd_lm>=0.5]=1
ans={'id': test_data['id'],'target':y_pred_grd_lm}
ans=DataFrame(ans)
print(ans)

             id    target
0             0  0.024865
1             1  0.025896
2             2  0.034001
3             3  0.020190
4             4  0.034001
...         ...       ...
892811  1488022  0.061072
892812  1488023  0.044780
892813  1488024  0.030664
892814  1488025  0.028388
892815  1488026  0.025085

[892816 rows x 2 columns]
